In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np

In [2]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [3]:
stop_times_full = pd.read_csv(gtfs_dir + 'stop_times_reduced.csv')
trips_weekday = pd.read_csv(gtfs_dir + 'trips_weekday.csv')

In [4]:
hr_list = [4]

In [5]:
stop_times_full['hr'] = np.where(stop_times_full['hr'] > 23, stop_times_full['hr'] - 24, stop_times_full['hr'])
stop_times_full_o = stop_times_full.copy()
stop_times_full_d = stop_times_full.copy()
stop_times_full_d['stop_sequence_new'] = stop_times_full_d['stop_sequence_new'] - 1
stop_times_od = stop_times_full_o.merge(stop_times_full_d, left_on = ['trip_id', 'stop_sequence_new'],
                           right_on = ['trip_id', 'stop_sequence_new'], suffixes = ['_o', '_d'])
stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'route_short_name', 'direction_id']])

In [6]:
stop_times_od.groupby(['hr_o', 'INT_ID_o', 'INT_ID_d']).count().reset_index().sort_values(
    by = 'trip_id', ascending = False)[['INT_ID_o', 'INT_ID_d', 'trip_id']].drop_duplicates(
    subset = ['INT_ID_o', 'INT_ID_d'], keep = 'first').head(10)

,INT_ID_o,INT_ID_d,trip_id
107013,304.0,305.0,84
46283,305.0,304.0,65
49123,13452476.0,228.0,61
38448,228.0,13452476.0,60
53043,13468120.0,13468053.0,51
53063,13468224.0,13468120.0,51
52974,13467722.0,13467654.0,50
52953,13467654.0,13467505.0,50
53034,13468053.0,13467971.0,50
38246,116.0,13445250.0,49


In [7]:
max_freq = 70

In [8]:
stop_times_od[(stop_times_od['INT_ID_o'] == 304) & (stop_times_od['INT_ID_d'] == 305)][['route_short_name']].drop_duplicates()

,route_short_name
50561,134
199941,129
209275,131
209949,9131
213980,132
219108,133
234778,16
278222,169
402693,3
515067,38


In [9]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,route_short_name,direction_id
0,32828689,1,1,5,15,13466640.0,2,5,16,13466531.0,506,0
1,32828689,2,2,5,16,13466531.0,3,5,16,13466677.0,506,0
2,32828689,3,3,5,16,13466677.0,4,5,17,13466754.0,506,0
3,32828689,4,4,5,17,13466754.0,5,5,18,13466699.0,506,0
4,32828689,5,5,5,18,13466699.0,6,5,19,14043045.0,506,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1213714,32970488,6,7,3,4,13461421.0,8,3,4,13461602.0,306,1
1213715,32970488,7,8,3,4,13461602.0,9,3,5,13461912.0,306,1
1213716,32970488,8,9,3,5,13461912.0,10,3,5,13462284.0,306,1
1213717,32970488,9,10,3,5,13462284.0,11,3,6,13462427.0,306,1


In [10]:
stop_times_od = stop_times_od[stop_times_od['hr_o'].isin(hr_list)]

In [11]:
route_map_links = stop_times_od.groupby(['INT_ID_o', 'INT_ID_d']
                     ).count().reset_index()[['INT_ID_o','INT_ID_d', 'trip_id']]
route_map_links = route_map_links.rename(columns = {'trip_id':'freq'})

In [12]:
# adding a dummy link so networkx scales properly
route_map_links = route_map_links.append(pd.DataFrame.from_records([[1,2, 70]], columns = ['INT_ID_o', 'INT_ID_d','freq']))
route_map_links = route_map_links.reset_index().drop(columns = ['index'])

In [13]:
route_map_links['scaled_freq'] = route_map_links['freq']/max_freq

In [14]:
route_map_links['INT_ID_o'] = route_map_links['INT_ID_o'].astype(int)
route_map_links['INT_ID_d'] = route_map_links['INT_ID_d'].astype(int)

In [15]:
route_map_links

,INT_ID_o,INT_ID_d,freq,scaled_freq
0,100,13467909,17,0.242857
1,101,100,14,0.200000
2,101,102,4,0.057143
3,101,13466729,2,0.028571
4,101,13466960,2,0.028571
...,...,...,...,...
3365,30109241,13467971,2,0.028571
3366,30109241,13468053,2,0.028571
3367,30113634,13446075,4,0.057143
3368,30113634,13446293,5,0.071429


In [16]:
route_map_links.sort_values(by = 'freq', ascending = False)

,INT_ID_o,INT_ID_d,freq,scaled_freq
3369,1,2,70,1.000000
0,100,13467909,17,0.242857
413,13443870,13443782,16,0.228571
3021,13467909,13467752,15,0.214286
1,101,100,14,0.200000
...,...,...,...,...
1154,13450601,13450111,1,0.014286
3051,13468216,13468148,1,0.014286
2209,13460071,13459538,1,0.014286
1153,13450594,13451388,1,0.014286


In [17]:
route_map_links['link'] =  route_map_links['INT_ID_o'].astype(str) + '-' + route_map_links['INT_ID_d'].astype(str)

In [18]:
route_map_links

,INT_ID_o,INT_ID_d,freq,scaled_freq,link
0,100,13467909,17,0.242857,100-13467909
1,101,100,14,0.200000,101-100
2,101,102,4,0.057143,101-102
3,101,13466729,2,0.028571,101-13466729
4,101,13466960,2,0.028571,101-13466960
...,...,...,...,...,...
3365,30109241,13467971,2,0.028571,30109241-13467971
3366,30109241,13468053,2,0.028571,30109241-13468053
3367,30113634,13446075,4,0.057143,30113634-13446075
3368,30113634,13446293,5,0.071429,30113634-13446293


In [19]:
G = nx.DiGraph()

In [20]:
nodes = list((route_map_links['INT_ID_o'].append(route_map_links['INT_ID_d'])).drop_duplicates())

In [21]:
G.add_nodes_from(nodes)

In [22]:
link_list = []
for index, row in route_map_links.iterrows():
    attributes = {'link_id':row['link'], 
                  'freq':row['freq'], 'scaled_freq': row['scaled_freq']}
    link = (row['INT_ID_o'], row['INT_ID_d'], attributes)
    link_list.append(link)

In [23]:
G.add_edges_from(link_list)

In [24]:
nx.write_gexf(G, 'networks/EM-RM-16-singleDi.gexf')

In [25]:
route_map_links.to_csv('networks/EM-RM-16-singleDi.csv', index = False)